In [1]:
import torch
import torchvision
import os
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
from shutil import copyfile
from PIL import Image
import math

from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import torch.optim as optim
import torchvision.transforms as standard_transforms

import numpy as np
import glob
import cv2

from data_loader import Rescale
from data_loader import RescaleT
from data_loader import RandomCrop
from data_loader import CenterCrop
from data_loader import ToTensor
from data_loader import ToTensorLab
from data_loader import SalObjDataset

import pytorch_ssim
import pytorch_iou
from utils import AverageMeter, VisdomLinePlotter
from paperBASNET.metrics_mod import eval_mae, prec_recall

import warnings
warnings.filterwarnings('ignore')

In [51]:
### CHANGE SETTINGS HERE, BECAUSE WE CANNOT LOAD IMAGES OF ALL 6 DATASETS!!!
### Change dataset folder name
dataset_name = 'ECSSD' ### 'DUTS-TE', 'PASCALS', 'DUTOMRON', 'HKU-IS', 'SOD', 'ECSSD'
### Change to dataset directory
data_dir = os.getenv("HOME") + '/Documents/Courses/EE298-CV/finalproj/datasets/dataset_test/' + dataset_name + '/'
### Place model.pth in "evaluation_models" folder
model_name = 'struct_arch_w_flip_bestaverelaxed_201.pth'
### If evaluating structural architecture, set this to false. Else set to true to evaluated original pretrained BASNET model
is_orig_basnet = False

In [52]:
if is_orig_basnet: from paperBASNET.model import BASNet
else: from model import BASNet

def train():
    if os.name == 'nt':
       data_dir =   'C:/Users/marky/Documents/Courses/saliency/datasets/dataset_test/' + dataset_name + '/'
    else:
        data_dir =   os.getenv("HOME") + '/Documents/Courses/EE298-CV/finalproj/datasets/dataset_test/' + dataset_name + '/'
    test_image_dir = 'images/'
    test_label_dir = 'masks/'

    image_ext = '.jpg'
    label_ext = '.png'

    # model_dir = "./saved_models/basnet_bsi_aug/"
    model_dir = "./evaluation_models/"
    resume_train = True ### Just testing
    # resume_model_path = model_dir + "bestMAE/" + model_name
    resume_model_path = model_dir + model_name
    last_epoch = 1
    epoch_num = 100000
    batch_size_train = 8
    batch_size_val = 1
    train_num = 0
    val_num = 0
    enableInpaintAug = False
    device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

    test_img_name_list = glob.glob(data_dir + test_image_dir + '*' + image_ext)
    print("data_dir + test_image_dir + '*' + image_ext: ", data_dir + test_image_dir + '*' + image_ext)

    test_lbl_name_list = []
    for img_path in test_img_name_list:
        img_name = img_path.split("/")[-1]
        aaa = img_name.split(".")
        bbb = aaa[0:-1]
        imidx = bbb[0]
        for i in range(1,len(bbb)):
            imidx = imidx + "." + bbb[i]
        test_lbl_name_list.append(data_dir + test_label_dir + imidx + label_ext)

    print("---")
    print("test images: ", len(test_img_name_list))
    print("test labels: ", len(test_lbl_name_list))
    print("---")

    test_num = len(test_img_name_list)
    salobj_dataset_test = SalObjDataset(
        img_name_list=test_img_name_list,
        lbl_name_list=test_lbl_name_list,
        transform=transforms.Compose([
            RescaleT(256),
            RandomCrop(224),
            ToTensorLab(flag=0)]),
            category="test",
            enableInpaintAug=enableInpaintAug)
    salobj_dataloader_test = DataLoader(salobj_dataset_test, batch_size=batch_size_val, shuffle=False, num_workers=1)

    # ------- 3. define model --------
    # define the net
    net = BASNet(3, 1)
    if resume_train:
        # print("resume_model_path:", resume_model_path)
        checkpoint = torch.load(resume_model_path)
        net.load_state_dict(checkpoint)
    if torch.cuda.is_available():
        net.to(device)

    # ------- 4. define optimizer --------
    print("---define optimizer...")
    optimizer = optim.Adam(net.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0)

    best_ave_mae = 100000
    best_max_fmeasure = 0
    best_relaxed_fmeasure = 0
    best_ave_maxf = 0
    best_own_RelaxedFmeasure=0
    average_mae = AverageMeter()
    average_maxf = AverageMeter()
    average_relaxedf = AverageMeter()
    average_own_RelaxedFMeasure = AverageMeter()
    average_prec = AverageMeter()
    average_rec = AverageMeter()
    print("---Evaluate model---")
    net.eval()
    max_epoch_fmeasure = 0
    for i, data in enumerate(salobj_dataloader_test):
        inputs, labels = data['image'], data['label']
        inputs = inputs.type(torch.FloatTensor)
        labels = labels.type(torch.FloatTensor)
        if torch.cuda.is_available(): inputs_v, labels_v = Variable(inputs.to(device), requires_grad=False), Variable(labels.to(device), requires_grad=False)
        else: inputs_v, labels_v = Variable(inputs, requires_grad=False), Variable(labels, requires_grad=False)
        if is_orig_basnet: d0, d1, d2, d3, d4, d5, d6, d7 = net(inputs_v)
        else: d0, d1, d2, d3, d4, d5, d6, d7, d1_struct, d2_struct, d3_struct, d4_struct, d5_struct, d6_struct, d7_struct = net(inputs_v)
        maeVal=eval_mae(d0.cpu().data,labels.cpu().data)
        maeVal = eval_mae(d0.cpu().data, labels.cpu().data).item()
        average_mae.update(maeVal, 1)
        prec, recall = prec_recall(d0.cpu().data, labels.cpu().data)
        average_prec.update(prec, 1)
        average_rec.update(recall, 1)
    beta2 = math.sqrt(0.3)  # for max F_beta metric
    denom = (beta2 ** 2 * average_prec.avg + average_rec.avg)
    if denom == 0: score = 0
    else: score = ((1 + beta2 ** 2) * average_prec.avg * average_rec.avg) / denom
    print("MAE score: ", average_mae.avg)
    print("MaxF score: ", score)

def run():
    torch.multiprocessing.freeze_support()
    print('loop')

In [5]:
### THIS IS A RESULT CELL: DO NOT MOFIFY THIS!!! TO GET RESULTS, USE ANOTHER CELL AND MODIFY GLOBAL SETTINGS. JUST COPY THE CODE BELOW...
### BASNET original - DUTS-TE
if __name__ == '__main__':
    run()
    train()

loop
data_dir + test_image_dir + '*' + image_ext:  /home/markytools/Documents/Courses/EE298-CV/finalproj/datasets/dataset_test/DUTS-TE/images/*.jpg
---
test images:  5019
test labels:  5019
---
---define optimizer...
---Evaluate model---
MAE score:  0.05440299030874471
MaxF score:  tensor(0.8506)


In [9]:
### THIS IS A RESULT CELL: DO NOT MOFIFY THIS!!! TO GET RESULTS, USE ANOTHER CELL AND MODIFY GLOBAL SETTINGS. JUST COPY THE CODE BELOW...
### BASNET original - PASCALS
if __name__ == '__main__':
    run()
    train()

loop
data_dir + test_image_dir + '*' + image_ext:  /home/markytools/Documents/Courses/EE298-CV/finalproj/datasets/dataset_test/PASCALS/images/*.jpg
---
test images:  850
test labels:  850
---
---define optimizer...
---Evaluate model---
MAE score:  0.08531180506867873
MaxF score:  tensor(0.8468)


In [13]:
### THIS IS A RESULT CELL: DO NOT MOFIFY THIS!!! TO GET RESULTS, USE ANOTHER CELL AND MODIFY GLOBAL SETTINGS. JUST COPY THE CODE BELOW...
### BASNET original - DUTOMRON
if __name__ == '__main__':
    run()
    train()

loop
data_dir + test_image_dir + '*' + image_ext:  /home/markytools/Documents/Courses/EE298-CV/finalproj/datasets/dataset_test/DUTOMRON/images/*.jpg
---
test images:  5166
test labels:  5166
---
---define optimizer...
---Evaluate model---
MAE score:  0.0678968143558747
MaxF score:  tensor(0.7898)


In [17]:
### THIS IS A RESULT CELL: DO NOT MOFIFY THIS!!! TO GET RESULTS, USE ANOTHER CELL AND MODIFY GLOBAL SETTINGS. JUST COPY THE CODE BELOW...
### BASNET original - HKU-IS
if __name__ == '__main__':
    run()
    train()

loop
data_dir + test_image_dir + '*' + image_ext:  /home/markytools/Documents/Courses/EE298-CV/finalproj/datasets/dataset_test/HKU-IS/images/*.jpg
---
test images:  1447
test labels:  1447
---
---define optimizer...
---Evaluate model---
MAE score:  0.04243251562905199
MaxF score:  tensor(0.9167)


In [21]:
### THIS IS A RESULT CELL: DO NOT MOFIFY THIS!!! TO GET RESULTS, USE ANOTHER CELL AND MODIFY GLOBAL SETTINGS. JUST COPY THE CODE BELOW...
### BASNET original - SOD
if __name__ == '__main__':
    run()
    train()

loop
data_dir + test_image_dir + '*' + image_ext:  /home/markytools/Documents/Courses/EE298-CV/finalproj/datasets/dataset_test/SOD/images/*.jpg
---
test images:  300
test labels:  300
---
---define optimizer...
---Evaluate model---
MAE score:  0.12773287756756568
MaxF score:  tensor(0.8408)


In [29]:
### THIS IS A RESULT CELL: DO NOT MOFIFY THIS!!! TO GET RESULTS, USE ANOTHER CELL AND MODIFY GLOBAL SETTINGS. JUST COPY THE CODE BELOW...
### BASNET original - ECSSD
if __name__ == '__main__':
    run()
    train()

loop
data_dir + test_image_dir + '*' + image_ext:  /home/markytools/Documents/Courses/EE298-CV/finalproj/datasets/dataset_test/ECSSD/images/*.jpg
---
test images:  1000
test labels:  1000
---
---define optimizer...
---Evaluate model---
MAE score:  0.04569519773882348
MaxF score:  tensor(0.9347)


In [36]:
### THIS IS A RESULT CELL: DO NOT MOFIFY THIS!!! TO GET RESULTS, USE ANOTHER CELL AND MODIFY GLOBAL SETTINGS. JUST COPY THE CODE BELOW...
### BASNET STRUCT ARCH W/ HFLIP - epoch201 (bestAveRelaxedF) - DUTS-TE
if __name__ == '__main__':
    run()
    train()

loop
data_dir + test_image_dir + '*' + image_ext:  /home/markytools/Documents/Courses/EE298-CV/finalproj/datasets/dataset_test/DUTS-TE/images/*.jpg
---
test images:  5019
test labels:  5019
---
---define optimizer...
---Evaluate model---
MAE score:  0.06770727324743372
MaxF score:  tensor(0.7932)


In [40]:
### THIS IS A RESULT CELL: DO NOT MOFIFY THIS!!! TO GET RESULTS, USE ANOTHER CELL AND MODIFY GLOBAL SETTINGS. JUST COPY THE CODE BELOW...
### BASNET STRUCT ARCH W/ HFLIP - epoch201 (bestAveRelaxedF) - PASCALS
if __name__ == '__main__':
    run()
    train()

loop
data_dir + test_image_dir + '*' + image_ext:  /home/markytools/Documents/Courses/EE298-CV/finalproj/datasets/dataset_test/PASCALS/images/*.jpg
---
test images:  850
test labels:  850
---
---define optimizer...
---Evaluate model---
MAE score:  0.09927098126890248
MaxF score:  tensor(0.8096)


In [43]:
### THIS IS A RESULT CELL: DO NOT MOFIFY THIS!!! TO GET RESULTS, USE ANOTHER CELL AND MODIFY GLOBAL SETTINGS. JUST COPY THE CODE BELOW...
### BASNET STRUCT ARCH W/ HFLIP - epoch201 (bestAveRelaxedF) - DUTOMRON
if __name__ == '__main__':
    run()
    train()

loop
data_dir + test_image_dir + '*' + image_ext:  /home/markytools/Documents/Courses/EE298-CV/finalproj/datasets/dataset_test/DUTOMRON/images/*.jpg
---
test images:  5166
test labels:  5166
---
---define optimizer...
---Evaluate model---
MAE score:  0.08090073526730117
MaxF score:  tensor(0.7462)


In [47]:
### THIS IS A RESULT CELL: DO NOT MOFIFY THIS!!! TO GET RESULTS, USE ANOTHER CELL AND MODIFY GLOBAL SETTINGS. JUST COPY THE CODE BELOW...
### BASNET STRUCT ARCH W/ HFLIP - epoch201 (bestAveRelaxedF) - HKU-IS
if __name__ == '__main__':
    run()
    train()

loop
data_dir + test_image_dir + '*' + image_ext:  /home/markytools/Documents/Courses/EE298-CV/finalproj/datasets/dataset_test/HKU-IS/images/*.jpg
---
test images:  1447
test labels:  1447
---
---define optimizer...
---Evaluate model---
MAE score:  0.05027070829439483
MaxF score:  tensor(0.8772)


In [50]:
### THIS IS A RESULT CELL: DO NOT MOFIFY THIS!!! TO GET RESULTS, USE ANOTHER CELL AND MODIFY GLOBAL SETTINGS. JUST COPY THE CODE BELOW...
### BASNET STRUCT ARCH W/ HFLIP - epoch201 (bestAveRelaxedF) - SOD
if __name__ == '__main__':
    run()
    train()

loop
data_dir + test_image_dir + '*' + image_ext:  /home/markytools/Documents/Courses/EE298-CV/finalproj/datasets/dataset_test/SOD/images/*.jpg
---
test images:  300
test labels:  300
---
---define optimizer...
---Evaluate model---
MAE score:  0.1181943744358917
MaxF score:  tensor(0.8336)


In [53]:
### THIS IS A RESULT CELL: DO NOT MOFIFY THIS!!! TO GET RESULTS, USE ANOTHER CELL AND MODIFY GLOBAL SETTINGS. JUST COPY THE CODE BELOW...
### BASNET STRUCT ARCH W/ HFLIP - epoch201 (bestAveRelaxedF) - ECSSD
if __name__ == '__main__':
    run()
    train()

loop
data_dir + test_image_dir + '*' + image_ext:  /home/markytools/Documents/Courses/EE298-CV/finalproj/datasets/dataset_test/ECSSD/images/*.jpg
---
test images:  1000
test labels:  1000
---
---define optimizer...
---Evaluate model---
MAE score:  0.05363992109440733
MaxF score:  tensor(0.9043)


In [58]:
### Results for the rest are in "EvaluationResults.png"